In [36]:
#!pip install PyMuPDF

In [2]:
import fitz
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import sys
import pandas as pd
import shutil
import sqlite3

In [3]:
pdf_path = os.path.join("data","Datos_abiertos_historicos_etv_2022.pdf")

In [4]:
def extract_links(pdf_path):
    links = []
    doc = fitz.open(pdf_path)    
    page = doc[0]
    for link in page.get_links():
        links.append(
            link['uri']
        )
    doc.close()
    return links

In [5]:
def descompresor(links):
    carpeta = os.path.join('data','csv-files')
    success = 0
    fail_download = 0
    fail_process = 0
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    ruta_bitacora = os.path.join('data','bitacora.txt')
    with open(ruta_bitacora, 'w') as log_file:

        #Redirigir la salida estándar al archivo
        sys.stdout = log_file
        for link in links:
            try:
                respuesta = requests.get(link)
                if respuesta.status_code == 200:
                    with ZipFile(BytesIO(respuesta.content)) as archivo_zip:
                        archivo_zip.extractall(carpeta)

                        # Cambiar el nombre del archivo
                        nombre_archivo = link[-32:-4]+'.csv'

                        for archivo in archivo_zip.namelist():
                            nuevo_nombre = os.path.join(carpeta, nombre_archivo)
                            os.rename(os.path.join(carpeta, archivo), nuevo_nombre)

                        print(f'Archivo descomprimido desde {link} en {carpeta}')
                        success+=1
                else:
                    print(f'Error al descargar {link}. Código de estado: {respuesta.status_code}')
                    fail_download+=1

            except Exception as e:
                print(f'Error al procesar {link}: {str(e)}')
                fail_process+=1
        print(f"Hubo {success} exitos, {fail_download} fallas al descargar, {fail_process} fallas al descomprimir")
        
        #Restaurar la salida estándar al archivo
        sys.stdout = sys.__stdout__

        print(f"Hubo {success} exitos, {fail_download} fallas al descargar, {fail_process} fallas al descomprimir")

    print(f'Los resultados se han guardado en {ruta_bitacora}')

In [6]:

enlaces = extract_links(pdf_path)

descompresor(enlaces)

In [7]:
cvs_files_path = os.path.join('data','csv-files')

In [8]:
def get_dataframe(carpeta):
    dataframe = pd.DataFrame()
    # Recorrer todos los archivos en la carpeta
    for archivo_csv in os.listdir(carpeta):
        if archivo_csv.endswith('.csv'):
            ruta_completa = os.path.join(carpeta, archivo_csv)
            df = pd.read_csv(ruta_completa)
            dataframe = pd.concat([dataframe, df])
    return dataframe

In [9]:
dengue_2022 = get_dataframe(cvs_files_path)
dengue_2022

Hubo 50 exitos, 2 fallas al descargar, 0 fallas al descomprimir
Los resultados se han guardado en data/bitacora.txt


,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2022-10-17,929942,1,47,18,20,2,2,18,20,...,2,2,2,2,5.0,1,5,3,18,20
1,2022-10-17,929943,1,34,25,18,2,2,25,18,...,2,2,2,2,5.0,1,5,3,25,18
2,2022-10-17,929944,2,50,23,2,2,2,23,2,...,2,2,2,2,5.0,1,5,3,23,2
3,2022-10-17,929946,2,23,30,193,2,2,30,193,...,1,2,2,2,5.0,1,5,3,30,193
4,2022-10-17,934897,2,23,12,29,2,2,12,29,...,2,2,2,2,5.0,2,5,1,12,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6896,2022-05-23,938113,1,10,27,15,2,2,27,15,...,2,2,2,2,5.0,2,5,1,27,15
6897,2022-05-23,938119,1,35,17,25,2,2,17,25,...,2,2,2,2,5.0,1,5,3,17,25
6898,2022-05-23,938122,1,32,27,4,2,2,27,4,...,2,2,2,2,5.0,1,5,3,27,4
6899,2022-05-23,938090,1,1,28,38,2,2,28,9,...,2,2,2,2,5.0,1,5,3,28,38


In [10]:
dengue_2022['FECHA_ACTUALIZACION'].unique() #Se indentifican formatos

array(['2022-10-17', '2022-03-14', '2022-01-31', '2022-09-21',
       '2022-11-22', '2022-03-07', '2022-10-31', '2022-09-26',
       '2022-03-22', '2022-12-12', '2022-08-22', '2022-07-04',
       '25/07/2022', '2022-02-08', '25/04/2022', '2022-04-18',
       '2022-09-05', '2022-11-07', '2022-08-29', '2022-11-28',
       '2022-08-01', '2022-10-10', '2022-01-06', '2022-06-13',
       '2022-02-28', '2022-12-05', '2022-03-28', '2022-07-11',
       '2022-01-10', '2022-08-15', '2022-05-09', '2022-04-11',
       '2022-06-06', '2022-01-17', '2022-09-13', '2022-06-20',
       '2022-04-04', '2022-05-16', '2022-07-18', '2022-06-27',
       '2022-11-14', '2022-08-09', '2022-05-30', '2022-05-02',
       '2022-10-24', '2022-01-24', '2022-02-14', '2022-05-23'],
      dtype=object)

In [11]:
#Se cambian a objetos datetime los formatos mixtos
date1 = pd.to_datetime(dengue_2022['FECHA_ACTUALIZACION'], errors='coerce', format='%Y-%m-%d')
date2 = pd.to_datetime(dengue_2022['FECHA_ACTUALIZACION'], errors='coerce', format='%d/%m/%Y')
dengue_2022['FECHA_ACTUALIZACION'] = date1.fillna(date2)
#Se organizan las fechas
dengue_2022 = dengue_2022.sort_values(by='FECHA_ACTUALIZACION')

In [12]:
dengue_2022

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
3861,2022-01-06,925362,1,44,5,28,2,2,5,20,...,2,2,2,2,5.0,1,5,3,5,28
13258,2022-01-06,907229,1,54,14,103,2,2,14,103,...,2,2,2,2,5.0,1,2,2,14,103
13259,2022-01-06,907191,2,23,12,67,2,2,12,22,...,2,2,2,2,5.0,1,5,2,12,67
13260,2022-01-06,907194,2,26,19,7,2,2,19,7,...,2,2,2,2,5.0,1,5,3,19,7
13261,2022-01-06,907223,1,43,19,7,2,2,19,7,...,2,2,2,2,5.0,1,1,2,19,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17789,2022-12-12,985187,2,9,26,26,2,2,26,42,...,2,2,2,2,5.0,1,5,1,26,26
17790,2022-12-12,985188,1,2,4,2,2,2,4,2,...,2,2,2,2,5.0,1,5,3,4,2
17791,2022-12-12,985191,2,32,26,42,2,2,26,42,...,2,2,2,2,5.0,2,5,1,26,42
17780,2022-12-12,985146,2,9,5,25,2,2,5,25,...,2,2,2,2,5.0,1,5,3,5,25


In [13]:
dengue_2022.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'SEXO', 'EDAD_ANOS',
       'ENTIDAD_RES', 'MUNICIPIO_RES', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'ENTIDAD_UM_NOTIF', 'MUNICIPIO_UM_NOTIF', 'INSTITUCION_UM_NOTIF',
       'FECHA_SIGN_SINTOMAS', 'TIPO_PACIENTE', 'HEMORRAGICOS', 'DIABETES',
       'HIPERTENSION', 'ENFERMEDAD_ULC_PEPTICA', 'ENFERMEDAD_RENAL',
       'INMUNOSUPR', 'CIRROSIS_HEPATICA', 'EMBARAZO', 'DEFUNCION', 'DICTAMEN',
       'TOMA_MUESTRA', 'RESULTADO_PCR', 'ESTATUS_CASO', 'ENTIDAD_ASIG',
       'MUNICIPIO_ASIG'],
      dtype='object')

In [14]:
dengue_2022.to_csv(os.path.join('data','dengue_2022.csv'))

In [15]:
def inserta_registros(df):
  # Creando una conección a la base de datos SQLite.
  connection = sqlite3.connect(os.path.join('database','enfermedades_db.db'))

  nombre_base_datos_original = os.path.join('database','enfermedades_db.db')

  # Nombre del archivo de respaldo
  nombre_base_datos_respaldo = os.path.join('database','enfermedades_db_respaldo.db')

  # Copiando el archivo de la base de datos para crear un respaldo
  shutil.copy2(nombre_base_datos_original, nombre_base_datos_respaldo)

  # Creando un objeto Cursor para ejecutar instrucciones en SQL.
  cursor = connection.cursor()

  cursor.execute("DELETE FROM registro")
  connection.commit()

  df.to_sql('registro', connection, index=False, if_exists='replace')

  connection.close()

In [16]:
folder_path_data = os.path.join('data','dengue_2022.csv')
dengue_data = pd.read_csv(folder_path_data)

inserta_registros(dengue_data)